In [ ]:
# This is a playground file where I want to implement search itself

import pandas as pd
import numpy as np

In [ ]:
# set global parameters (perlude)
const_number_pages = 12

webpage_dict = {'A':['Anchovy','Grouper', 'Mackerel', 'Pike', 'Salmon', 'Betta', 'Walleye'],
               'B': ['Grouper', 'Pike', 'Wahoo', 'Tuna', 'Shad', 'Cod', 'Carp', 'Opah', 'Herring'],
               'C':['Anchovy', 'Wahoo', 'Tuna', 'Opah', 'Mudskipper'],
               'D':['Mackerel', 'Pike', 'Wahoo', 'Salmon', 'Mudskipper'],
               'E':['Anchovy', 'Grouper', 'Shad', 'Mudskipper', 'Herring', 'Walleye'],
               'F':['Grouper', 'Cod', 'Betta', 'Carp', 'Perch', 'Mudskipper'],
               'G':['Mackerel', 'Tuna', 'Shad', 'Cod', 'Opah', 'Walleye'],
               'H':['Anchovy', 'Mackerel', 'Pike', 'Salmon', 'Perch', 'Herring'],
               'I':['Pike', 'Tuna', 'Cod', 'Salmon', 'Carp', 'Walleye'],
               'J':['Grouper', 'Mackerel', 'Tuna', 'Perch', 'Walleye'],
               'K':['Mackerel', 'Wahoo', 'Shad', 'Betta', 'Carp', 'Opah', 'Herring'],
               'L':['Grouper', 'Wahoo', 'Betta', 'Perch', 'Mudskipper', 'Herring']}

out_edge_network = {'A':['C','E'],
               'B':['A','C'],
               'C':[],
               'D':['B'],
               'E':['C','F','G','I'],
               'F':['C', 'B','D','H'],
               'G':['F','I'],
               'H':['D','G','J'],
               'I':['G','J','K'],
               'J':['G','H','L'],
               'K':['J'],
               'L':['J']}



webpage_names = {
    "A": "Tim’s Wet Pets",
    "B": "Finny’s Fried Fish",
    "C": "By the Sea",
    "D": "Scales and Tails", 
    "E": "Fillets ’R Us", 
    "F": "It’s Better Wetter" ,
    "G": "Friendly Fish",  
    "H": "Frank’s Aquariums", 
    "I": "Fishmonger’s Bane", 
    "J": "Fish Eye Cafe", 
    "K": "Happy Fins", 
    "L": "Fishtastic" 
}

# pretty matrix print
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")
        
def print_websites(ranking_vector: list) -> None:
    for i, j in enumerate(ranking_vector):
        try:
            print(f"{i+1}. {j}: {webpage_names[j]}")
        except KeyError:
            print("Wrong vector")
            return